In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

testVeriSeti = datasets.ImageFolder('/content/drive/My Drive/YapayZekaRessam/test', transform=transform)
testYukleyici = DataLoader(testVeriSeti, batch_size=32, shuffle=False, num_workers=4)


sinifAdlari = testVeriSeti.classes
print('Test verileri hazırlandı.')

Test verileri hazırlandı.


In [ ]:
import torch
import torch.nn as nn
from timm import create_model

cihaz = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Cihaz: {cihaz}')

model = create_model('vit_base_patch16_224', pretrained=False)
model.head = nn.Linear(model.head.in_features, 10)
model = model.to(cihaz)

model_yolu = '/content/drive/My Drive/colabmodels/vitmodelson.pth'
kayit_dict = torch.load(model_yolu)

model.load_state_dict(kayit_dict['model_state_dict'])

print('ViT modeli başarıyla yüklendi ve hazır.')

Cihaz: cuda


<ipython-input-3-3824091a3c27>:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  kayit_dict = torch.load(model_yolu)


ViT modeli başarıyla yüklendi ve hazır.


In [ ]:
import io
import ipywidgets as widgets
from IPython.display import display, Image as IPImage, clear_output
from PIL import Image

yukleme_butonu = widgets.FileUpload(accept='image/*', multiple=False)
cikti = widgets.Output()

display(yukleme_butonu)
display(cikti)

def tahmin_et(degisim):
    global cikti
    with cikti:
        clear_output()
        yuklenen_dosya = yukleme_butonu.value
        if len(yuklenen_dosya) > 0:
            img_veri = list(yuklenen_dosya.values())[0]['content']
            img = Image.open(io.BytesIO(img_veri)).convert('RGB')
            img_tensor = transform(img).unsqueeze(0).to(cihaz)

            model.eval()
            with torch.no_grad():
                cikti = model(img_tensor)
                _, tahmin = torch.max(cikti, 1)

            display(IPImage(data=img_veri, format='png'))
            print(f'Tahmin edilen kategori: {sinifAdlari[tahmin.item()]}')

yukleme_butonu.observe(tahmin_et, names='value')

FileUpload(value={}, accept='image/*', description='Upload')

Output()